In [1]:
import pandas as pd
import warnings
import numpy as np
import random
from user_profile.profile_service import *
warnings.filterwarnings('ignore')

INPUT_FILE = 'data/xhs-T2-40W.csv'
CONFIG_FILE = 'config/config.yaml'
DOMAIN = 'POSTPARTUM'

In [2]:
# load config
config = load_config(CONFIG_FILE)

global_nickname_blacklist = get_global_nickname_blacklist(config)
domain_nickname_blacklist = get_domain_nicknames_blacklist(config, domain=DOMAIN)

domain_content_whitelist = get_content_whitelist(config, domain=DOMAIN )
domain_content_blacklist = get_content_blacklist(config, domain=DOMAIN )

In [3]:
# load data then remove global biz user
df = load_data(INPUT_FILE)
df = remove_biz_user(df, global_nickname_blacklist)
df = remove_biz_user(df, domain_nickname_blacklist)
len(df)

172970

In [4]:
df

,_id,source,note_id,nickname,user_id,type,content,collected_count,comment_count,share_count,...,interactTag,interactNum,constellation,career,field,education,cert,if_exists_comment,sale_flag,content_length
0,"{""$oid"":""66851277beabdee785c2fbd7""}",user_post_month_up,659cbd18000000001303409a,熙木庚泥,63219c64000000002303a7a1,normal,这个案例已经落地，分享一下设计的思路，现场之后先行的，是控制性的空间布局先行，给场地匹配到合...,0,0,0,...,0-1k,0,NaN,NaN,NaN,NaN,NaN,0,0,132
2,"{""$oid"":""6694e1bb3879c600a342bc37""}",user_post_day_up,61b5f7f50000000021039a09,甜梨儿,59a0dd4c5e87e73e98e9a385,normal,疫情如此严重的同时，为什么有这么多人进来，如果是隔离回来也不可能，现在疫情这么严重正是爆发的...,0,6,0,...,1k-1w,1000,NaN,NaN,NaN,NaN,NaN,0,0,65
3,"{""$oid"":""6694ea4c5a33aee765bed978""}",user_post_day_up,61b62bfc000000000102c40d,傲娇陳,59aa445b82ec392842abfe6b,normal,幸运的是 爱与被爱同事发生了❤️\n在营地吃了烧烤和火锅后被叫出去啦 虽然早有预感 但是还是...,4,4,10,...,1k-1w,1000,NaN,NaN,NaN,NaN,NaN,0,0,69
7,"{""$oid"":""669473b93879c600a34145cb""}",user_post_day_up,61b73735000000002103e4c7,尼采啊上帝真的死了,5964fe4b5e87e747b584e9fe,normal,在上好佳，薯片都弱爆了好吗！\n小学就爱吃这个可可甜心，外表酥酥脆脆，一口咬下去，里面都是巧...,7,10,10,...,1k-1w,1000,NaN,NaN,NaN,NaN,NaN,0,0,147
8,"{""$oid"":""6693f7fd5a18ba772648cc74""}",user_post_day_up,61b743b1000000002103b8d7,小猫,58fc6c845e87e71f3fc48c2f,normal,双十一后不清点一下都不知自己究竟囤了多少。我是真珠肌野生代言人[笑哭R]因为面霜双十一真的是...,0,0,0,...,0-1k,100,NaN,NaN,NaN,NaN,NaN,0,0,71
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
296956,"{""$oid"":""668ffdfc9253eeb5f3c0b64a""}",user_post_day_up,668f801400000000250060ed,娄娄,5a375b5ee8ac2b3fc6449dd6,normal,培训期间镇海随便找的一家店，饭是真的香…,0,0,0,...,0-1k,0,水瓶座,NaN,NaN,NaN,NaN,0,0,20
296957,"{""$oid"":""668ffe2c5a33aee765aef3c3""}",user_post_day_up,668f80b400000000250007f6,娄娄,5a375b5ee8ac2b3fc6449dd6,normal,前段时间某音总是刷到，去打卡了，现在回来看到这图，就是想不起店名…味道还不如古田村,0,0,0,...,0-1k,0,水瓶座,NaN,NaN,NaN,NaN,0,0,41
296958,"{""$oid"":""668ffe5e3879c600a32aa022""}",user_post_day_up,668f814e000000002501656b,娄娄,5a375b5ee8ac2b3fc6449dd6,normal,这次属于二刷了，他家乳鸽惊艳我了\n其他菜中规中矩吧，说不得好吃或者不好吃,0,0,0,...,0-1k,0,水瓶座,NaN,NaN,NaN,NaN,0,0,36
296960,"{""$oid"":""668ffea93439d3ed04d9a80e""}",user_post_day_up,668f826c000000002500318d,娄娄,5a375b5ee8ac2b3fc6449dd6,normal,咖喱配米饭错不了\n也是二刷了，更加喜欢来福士的泰妃店,0,0,0,...,0-1k,0,水瓶座,NaN,NaN,NaN,NaN,0,0,26


In [4]:
ndf = df.copy()
# filter by score based on content white/black list and score threshold
df_filtered = filter_by_score(ndf, domain_content_whitelist, domain_content_blacklist,3)
df_user = group_by_user(df_filtered)
print(f'renovate - #posts:{len(df_filtered)}, #users: {len(df_user)}')

renovate - #posts:157, #users: 43


In [10]:
df[df['user_id']=='568e7eaeb8c8b42233ef651c']

,_id,source,note_id,nickname,user_id,type,content,collected_count,comment_count,share_count,...,age,interactTag,interactNum,constellation,career,field,education,cert,if_exists_comment,sale_flag


In [7]:
# check single post with user id
user_id = '545a18cfd6e4a94f3804336b'
show_content_by_user_id(df_filtered, domain_content_whitelist, domain_content_blacklist, user_id)

IndexError: index 0 is out of bounds for axis 0 with size 0

In [8]:
# it's gonna take a while
# try df_user for pets, df_user_dog for dog, df_user_cat for cat
show_all_contents(df_user[:30], domain_content_whitelist, domain_content_blacklist)

========== user[0]: 怂弱弱, score: 15.0
孕晚期两天半极限羊水上涨📈｜胎盘低置36w+5突然查出来羊水偏少 一开始还没意识到问题的严重性 看到同期产检的指数到7就要住院了 还有需要催产打针的 立马紧张起来
小红书搜了不少补羊水的帖子 努力实践了
首先这两天都努力多喝白水了 至少保证2L
然后一天一瓶电解质水 一个新鲜椰子
上午打豆浆2小时内喝掉（20g黄豆 50g山药 5.6粒枸杞打豆浆 一般加水到1100ml左右 豆浆打好后再加入600ml牛奶左右）
没事就多喝水
这样两天下来 今天37w复查羊水指数顺利过关
剩下的2周多要继续坚持
	
上次产检因为羊水偏少也没有和🧑‍⚕️约时间
今天和医生约了6.26 39周剖
虽然上一次b超胎盘位置有上涨 但彩超大夫当时也提到说因为孩子长大的缘故胎盘位置有遮挡 再一个还是觉得3.9cm也不是很稳妥 外加也的确恐惧顺产中的不确定性以及侧切和撕裂
所以还是和🧑‍⚕️约了剖 如果在39周前发动了 那就顺其自然啦 希望一切顺顺利利！
#产检[话题]# #b超[话题]# #足月[话题]# #孕晚期倒计时[话题]# #卸货倒计时[话题]# #羊水[话题]# #补羊水[话题]# #胎盘低置[话题]#产检;b超;足月;孕晚期倒计时;卸货倒计时;羊水;补羊水;胎盘低置||一个不太胖的🐉宝宝 胎盘下缘距子宫内口2cm宝宝好像一直不太胖 双顶径一直不大
今天刚做了32w+5的B超 感觉宝宝有点偏小 但🧑‍⚕️说是不小正常
胎盘位置还是那样 胎盘下缘距子宫内口约2.08cm
感觉一直就在2左右徘徊
这种情况是不是只能剖了[笑哭R]
#b超[话题]# #彩超[话题]# #产检[话题]# #胎儿[话题]# #孕期日常[话题]# #胎盘低[话题]# #胎盘低置[话题]#b超;彩超;产检;胎儿;孕期日常;胎盘低;胎盘低置||7月预产期 喜提近7个月的假期7.3预产期 本来打算6.24开始休产假
上周产检发现羊水偏少 胎盘位置还是没怎么上涨
医生给开了两周假条 约了6.26剖
这样相当于今年的班就上到6.7啦
6.11-6.21病假 其中两天超了全薪病假天数 扣点工资
6.24打算开始休产假
剖腹产的话 到12月份上旬 再用今年剩的13天年假 刚好可以到2025年上班 育儿假留到明年用
顺的话 到11月底 用掉一天生日假+13天